In [14]:
!pip uninstall keras
!pip install keras==2.12.0
!pip uninstall tensorflow
!pip install tensorflow==2.12.0

Found existing installation: keras 2.15.0
Uninstalling keras-2.15.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/keras-2.15.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/keras/*
Proceed (Y/n)? y
  Successfully uninstalled keras-2.15.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 2.12.0 which is incompatible.


Found existing installation: tensorflow 2.15.0
Uninstalling tensorflow-2.15.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.10/dist-packages/tensorflow-2.15.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/tensorflow/*
Proceed (Y/n)? y
  Successfully uninstalled tensorflow-2.15.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 79.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 32.8 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling tensorflow-estimator-2.15.0:
      Successfully uninstalled tens

In [1]:
from sklearn.metrics import accuracy_score


In [6]:
!pip install scikeras
from scikeras.wrappers import KerasClassifier
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers import Dense, Dropout

# Load the dataset
df = pd.read_csv('/content/train.csv')

# Impute missing values
mean_age = df['Age'].mean()
df['Age'].fillna(mean_age, inplace=True)
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)

# Drop unnecessary columns
df = df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp', 'Parch'])

# One-hot encoding for 'Embarked'
df = pd.get_dummies(df, columns=['Embarked'], prefix='Embarked')

# Encode categorical features
le = LabelEncoder()
df['Sex'] = le.fit_transform(df['Sex'])

# Separate features and target variable
targets = df['Survived']
df = df.drop(columns='Survived')

# Split the dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(df, targets, test_size=0.3, random_state=42)

# Apply SMOTE for oversampling
smote = SMOTE()
x_train_resampled, y_train_resampled = smote.fit_resample(x_train, y_train)

# Standardize features for PCA
scaler = StandardScaler()
x_train_resampled_scaled = scaler.fit_transform(x_train_resampled)
x_test_scaled = scaler.transform(x_test)

# Apply PCA to reduce dimensionality
pca = PCA(n_components=0.95)  # Keep 95% of variance
x_train_pca = pca.fit_transform(x_train_resampled_scaled)
x_test_pca = pca.transform(x_test_scaled)

# Build the neural network model
model_nn = Sequential()
model_nn.add(Dense(128, input_dim=x_train_pca.shape[1], activation='relu'))
model_nn.add(Dropout(0.5))
model_nn.add(Dense(64, activation='relu'))
model_nn.add(Dropout(0.5))
model_nn.add(Dense(32, activation='relu'))
model_nn.add(Dense(1, activation='sigmoid'))
model_nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the neural network model
model_nn.fit(x_train_pca, y_train_resampled, epochs=100, batch_size=32, verbose=1)

# Make predictions on the test set using the neural network
nn_pred_prob = model_nn.predict(x_test_pca)
nn_pred = (nn_pred_prob > 0.5).astype(int)

# Evaluate the neural network model
print("Neural Network Classification Report:\n", classification_report(y_test, nn_pred))

# Build and train Gradient Boosting model
model_gb = GradientBoostingClassifier()
model_gb.fit(x_train_resampled_scaled, y_train_resampled)

# Make predictions on the test set using Gradient Boosting
gb_pred = model_gb.predict(x_test_scaled)

# Evaluate the Gradient Boosting model
print("Gradient Boosting Classification Report:\n", classification_report(y_test, gb_pred))

# Build and train Random Forest model
model_rf = RandomForestClassifier()
model_rf.fit(x_train_resampled_scaled, y_train_resampled)

# Make predictions on the test set using Random Forest
rf_pred = model_rf.predict(x_test_scaled)

# Evaluate the Random Forest model
print("Random Forest Classification Report:\n", classification_report(y_test, rf_pred))

Epoch 1/100
25/25 [==============================] - 1s 3ms/step - loss: 0.6649 - accuracy: 0.6378
Epoch 2/100
25/25 [==============================] - 0s 3ms/step - loss: 0.5794 - accuracy: 0.7296
Epoch 3/100
25/25 [==============================] - 0s 3ms/step - loss: 0.5030 - accuracy: 0.7704
Epoch 4/100
25/25 [==============================] - 0s 3ms/step - loss: 0.4528 - accuracy: 0.7985
Epoch 5/100
25/25 [==============================] - 0s 2ms/step - loss: 0.4830 - accuracy: 0.7793
Epoch 6/100
25/25 [==============================] - 0s 3ms/step - loss: 0.4379 - accuracy: 0.7985
Epoch 7/100
25/25 [==============================] - 0s 3ms/step - loss: 0.4699 - accuracy: 0.7883
Epoch 8/100
25/25 [==============================] - 0s 3ms/step - loss: 0.4538 - accuracy: 0.7908
Epoch 9/100
25/25 [==============================] - 0s 3ms/step - loss: 0.4389 - accuracy: 0.8074
Epoch 10/100
25/25 [==============================] - 0s 2ms/step - loss: 0.4491 - accuracy: 0.7934
Epoch 11/

In [8]:
print("Shape of x_train_pca:", x_train_pca.shape)


# Evaluate the model on the test set
test_loss, test_accuracy = model_nn.evaluate(x_test_pca, y_test)

# Display the accuracy
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Shape of x_train_pca: (784, 6)
9/9 [==============================] - 0s 4ms/step - loss: 0.4749 - accuracy: 0.7948
Test Accuracy: 79.48%


In [2]:
!pip install scikeras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier


In [3]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

# Load the dataset
df = pd.read_csv('/content/train.csv')

# Impute missing values
mean_age = df['Age'].mean()
df['Age'].fillna(mean_age, inplace=True)
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)

# Drop unnecessary columns
df = df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp', 'Parch'])

# One-hot encoding for 'Embarked'
df = pd.get_dummies(df, columns=['Embarked'], prefix='Embarked')

# Encode categorical features
le = LabelEncoder()
df['Sex'] = le.fit_transform(df['Sex'])

# Separate features and target variable
targets = df['Survived']
df = df.drop(columns='Survived')

# Split the dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(df, targets, test_size=0.3, random_state=42)

# Apply SMOTE for oversampling
smote = SMOTE()
x_train_resampled, y_train_resampled = smote.fit_resample(x_train, y_train)

# Standardize features for PCA
scaler = StandardScaler()
x_train_resampled_scaled = scaler.fit_transform(x_train_resampled)
x_test_scaled = scaler.transform(x_test)

# Apply PCA to reduce dimensionality
pca = PCA(n_components=0.95)  # Keep 95% of variance
x_train_pca = pca.fit_transform(x_train_resampled_scaled)
x_test_pca = pca.transform(x_test_scaled)

# Build the neural network model
def create_nn_model():
    model = Sequential()
    model.add(Dense(128, input_dim=x_train_pca.shape[1], activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Wrap Keras model so that it can be used by scikit-learn
model_nn = KerasClassifier(build_fn=create_nn_model, epochs=100, batch_size=32, verbose=0)

# Fine-tune neural network model using GridSearchCV
param_grid = {
    'epochs': [50, 100, 150],
    'batch_size': [16, 32, 64]
}

grid_search = GridSearchCV(estimator=model_nn, param_grid=param_grid, scoring='accuracy', cv=3)
grid_result = grid_search.fit(x_train_pca, y_train_resampled)

# Get the best parameters
best_epochs = grid_result.best_params_['epochs']
best_batch_size = grid_result.best_params_['batch_size']

# Build and train the final neural network model with the best parameters
final_model_nn = create_nn_model()
final_model_nn.fit(x_train_pca, y_train_resampled, epochs=best_epochs, batch_size=best_batch_size, verbose=1)

# Make predictions on the test set using the final neural network
nn_pred_prob = final_model_nn.predict(x_test_pca)
nn_pred = (nn_pred_prob > 0.5).astype(int)

# Evaluate the final neural network model
print("Neural Network Classification Report:\n", classification_report(y_test, nn_pred))

# Build and train Gradient Boosting model
model_gb = GradientBoostingClassifier()
model_gb.fit(x_train_resampled_scaled, y_train_resampled)

# Make predictions on the test set using Gradient Boosting
gb_pred = model_gb.predict(x_test_scaled)

# Evaluate the Gradient Boosting model
print("Gradient Boosting Classification Report:\n", classification_report(y_test, gb_pred))

# Build and train Random Forest model
model_rf = RandomForestClassifier()
model_rf.fit(x_train_resampled_scaled, y_train_resampled)

# Make predictions on the test set using Random Forest
rf_pred = model_rf.predict(x_test_scaled)

# Evaluate the Random Forest model
print("Random Forest Classification Report:\n", classification_report(y_test, rf_pred))


<ipython-input-3-c44bbb4417da>:65: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model_nn = KerasClassifier(build_fn=create_nn_model, epochs=100, batch_size=32, verbose=0)


9/9 [==============================] - 0s 2ms/step
Epoch 1/100
13/13 [==============================] - 1s 3ms/step - loss: 0.6815 - accuracy: 0.5459
Epoch 2/100
13/13 [==============================] - 0s 3ms/step - loss: 0.6400 - accuracy: 0.6480
Epoch 3/100
13/13 [==============================] - 0s 3ms/step - loss: 0.5780 - accuracy: 0.7360
Epoch 4/100
13/13 [==============================] - 0s 3ms/step - loss: 0.5279 - accuracy: 0.7577
Epoch 5/100
13/13 [==============================] - 0s 3ms/step - loss: 0.4963 - accuracy: 0.7717
Epoch 6/100
13/13 [==============================] - 0s 3ms/step - loss: 0.4843 - accuracy: 0.7832
Epoch 7/100
13/13 [==============================] - 0s 3ms/step - loss: 0.4817 - accuracy: 0.7768
Epoch 8/100
13/13 [==============================] - 0s 3ms/step - loss: 0.4773 - accuracy: 0.7934
Epoch 9/100
13/13 [==============================] - 0s 3ms/step - loss: 0.4490 - accuracy: 0.7921
Epoch 10/100
13/13 [==============================] - 0s 3

In [4]:
# Evaluate the final neural network model
print("Neural Network Classification Report:\n", classification_report(y_test, nn_pred))

# Alternatively, if you want to calculate the accuracy manually
accuracy = accuracy_score(y_test, nn_pred)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Neural Network Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.89      0.84       157
           1       0.81      0.67      0.73       111

    accuracy                           0.80       268
   macro avg       0.80      0.78      0.79       268
weighted avg       0.80      0.80      0.79       268

Test Accuracy: 79.85%
